In [1]:
!pip install Flask
!pip install langchain

In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -U langchain-community
!pip install -U jupyter ipywidgets
# cần để xử lý các thành phần trong vectorstores như Chroma
# hệ thống thiếu các gói hỗ trợ Jupyter để hiển thị tiến trình tqdm.

In [4]:
from flask import Flask, render_template, request
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
from langchain.chains.question_answering import load_qa_chain  
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [6]:
prompt_template='''
Use the following pieces of information to answer the user's question.
If you don't know the answer,just say that you don't know,don't try to makeup the answer.
Context: {context}
Question: {question}

Only return the helpful aswer below and nothing else.
Helpful Answer:
'''

In [7]:
app = Flask(__name__)


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb=Chroma(persist_directory='db',embedding_function=embeddings)


PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(
    model="TheBloke/Llama-2-7B-Chat-GGML",
    model_type="llama",
    config={'max_new_tokens':2048,'context_length' : 1024,'temperature':0.8}
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)



@app.route("/")
def index():
    return render_template('chat.html')



@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.form["msg"]
    input = msg
    print(input)
    result=qa({"query": input})
    print("Response : ", result["result"])
    return str(result["result"])



if __name__ == '__main__':
    app.run(host="0.0.0.0", port= 8080)

C:\Users\VIP\AppData\Local\Temp\ipykernel_26316\1901864551.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\SourceCode\GIT _ PRO\Cooking-Chatbot\COOKING-CHATBOT\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\VIP\AppData\Local\Temp\ipykernel_26316\1901864551.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.14:8080
Press CTRL+C to quit
